In [1]:
%env OMP_NUM_THREADS=1
%env MKL_NUM_THREADS=1
%env OPENBLAS_NUM_THREADS=1
%matplotlib nbagg
import matplotlib.pyplot as plt

env: OMP_NUM_THREADS=1
env: MKL_NUM_THREADS=1
env: OPENBLAS_NUM_THREADS=1


In [3]:
import numpy as np
from libertem import api
from libertem.executor.inline import InlineJobExecutor
from libertem.common.buffers import zeros_aligned
from libertem.udf.base import UDF

In [5]:
inline = InlineJobExecutor()
ctx = api.Context(executor=None)
# ctx = api.Context(executor=InlineJobExecutor())

In [6]:
ds = ctx.load("mib", path="/home/clausen/Data/Merlin/strain_karina/MOSFET/20181119 184223/default.hdr")

In [7]:
class MaxUDF(UDF):
    def get_result_buffers(self):
        return {
            'maxbuf': self.buffer(
                kind="sig", dtype="float32"
            )
        }
    
    def get_task_data(self, meta):
        return {
            'tmpbuf': self._zeros(ds.shape.sig, dtype="float32"),
        }
    
    def _zeros(self, *args, **kwargs):
        if self.params.aligned:
            return zeros_aligned(*args, **kwargs)
        else:
            return np.zeros(*args, **kwargs)
 
    def process_frame(self, frame):
        if self.params.keep_buffer:
            fprime = self.task_data.tmpbuf
        else:
            fprime = self._zeros(ds.shape.sig, dtype="float32")
        np.multiply(frame, frame, out=fprime)
        self.results.maxbuf[:] = np.maximum(fprime, self.results.maxbuf)
 
    def merge(self, dest, src):
        dest['maxbuf'][:] = np.maximum(dest['maxbuf'], src['maxbuf'])

In [8]:
%%timeit
res = ctx.run_udf(
   udf=MaxUDF(keep_buffer=True, aligned=True),
   dataset=ds,
)

7.72 s ± 612 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
res = ctx.run_udf(
   udf=MaxUDF(keep_buffer=False, aligned=True),
   dataset=ds,
)

9.96 s ± 102 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
res = ctx.run_udf(
   udf=MaxUDF(keep_buffer=True, aligned=False),
   dataset=ds,
)

6.76 s ± 92.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
res = ctx.run_udf(
   udf=MaxUDF(keep_buffer=False, aligned=False),
   dataset=ds,
)

7.16 s ± 151 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
